In [1]:
# relevant libraries for EDA
import graph
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import quandl
import seaborn as sns
import yfinance as yf

from datetime import datetime
from IPython.core.display import Image, display
from matplotlib import animation
from pathlib import Path

# reading in the Total Requirements data
data_path = f'{str(Path(os.getcwd()).parent)}/data/raw/AllTablesIO_Req'
H_matrices = {}
raw_2020 = pd.read_excel(f'{data_path}/download.xls')
H_2020 = raw_2020.iloc[6:-2, 2:]
H_2020.index = raw_2020.iloc[6:-2, 1]
H_2020.columns = raw_2020.iloc[5, 2:]
H_matrices['2020'] = H_2020
for i in range(1, 23):
    year = str(2020 - i)
    raw_data = pd.read_excel(f'{data_path}/download-{i + 1}.xls')
    H = raw_data.iloc[6:-2, 2:]
    H.index = raw_data.iloc[6:-2, 1]
    H.columns = raw_data.iloc[5, 2:]
    H_matrices[year] = H

Proposition 1 tells us that

$$\nabla\ln(y)=H\nabla z$$

Theoretically speaking if an idiosyncratic shock hits the 'Fabricated metal products' industry, then 'Construction', 'Motor vehicle and parts dealers' and related industries should move in unison. Let us test this simple hypothesis by specifying $\nabla z$ to be $-0.3$ in the 'Fabricated metal products' index. 

In [2]:
dz = np.zeros(71)
fab_metals_idx = H_matrices['2020'].columns.tolist().index('Fabricated metal products')
dz[fab_metals_idx] = -0.5
d_ln_y = np.matmul(H_matrices['2020'].iloc[:-1, :], dz)
d_ln_y.sort_values()[:20]

Unnamed: 1
Fabricated metal products                                              -0.564336
Primary metals                                                         -0.146208
Wholesale trade                                                        -0.056188
Miscellaneous professional, scientific, and technical services         -0.032584
Administrative and support services                                    -0.026251
Management of companies and enterprises                                -0.025008
Other real estate                                                      -0.019182
Chemical products                                                      -0.018818
Truck transportation                                                   -0.014266
Computer and electronic products                                       -0.011475
Utilities                                                              -0.011352
Machinery                                                              -0.010397
Federal Reserve b

In [3]:
def perturb(H, shock, industries):
    """
    :param: H - dataframe - total requirements matrix (size 71 x 71)
    :param: shock: float - the magnitude of the shock 
    :param: industries: list of strings - list of industries that are affected by the shock
    :return: pandas series of industries resulting shock sizes
    """
    dz = np.zeros(71)
    industry_idxs = [H.columns.tolist().index(industry) for industry in industries]
    dz[industry_idxs] = shock
    d_ln_y = np.matmul(H.iloc[:-1, :], dz)
    return d_ln_y

In [4]:
perturb(H=H_matrices['2020'], shock=-0.3, industries=['asdf']).sort_values(ascending=True)[:20]

# USEless Garbage - Ignore for now

# Simulating the Economy

## Basic Background

Wassily Leontief claimed an economy is divided into sectors and each sector produces its own product. To produce a product, an industry must use other sector products (potentially including itself - think Oil). In pseudo-equation format, we mean that

$$\text{Total Amount Produced} = \text{Total Internal Demand} + \text{Total External Demand.}$$

Let us consider an IO table withe the following toy dynamics:

$$\begin{cases}\text{Lumber Produced} =0.1\times\text{Lumber Produced} + 0.2\times\text{Oil Produced} + 0.2\times\text{Housing Produced} \\ \text{Oil Produced} = 0.15\times\text{Lumber Produced} + 0\times\text{Oil Produced} + 0.4\times\text{Housing Produced} \\ \text{Housing Produced} = 0.12\times\text{Lumber Produced} + 0.3\times\text{Oil Produced} + 0.2\times\text{Housing Produced}\end{cases}$$

or more succinctly, we have in matrix notation that the system can be written as 

$$C=\begin{bmatrix} 0.1 & 0.2 & 0.2 \\ 0.15 & 0 & 0.4 \\ 0.12 & 0.3 & 0.2\end{bmatrix}$$

which is defined as the consumption matrix. This gives us a sense of what products are required to produce one unit of the output which is defined as the rows of the matrix and the columns denote the industries needed by that row to produce a unit. Additionally, we may have products outside of the industry that each row may require to make produce - say $[d_1, d_2, d_3]^T$ (known as **external demand**) so that our total amount produced in this economy is given by the Leontief Input-Output model:

$$\begin{bmatrix}p_1 \\ p_2 \\ p_3\end{bmatrix} = \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33}\end{bmatrix}\begin{bmatrix} p_1 \\ p_2 \\ p_3\end{bmatrix} + \begin{bmatrix} d_1 \\ d_2 \\ d_3\end{bmatrix}$$

or more concisely

$$\hat{p} = C\hat{p} + \hat{d}.$$

Then we rearrange terms to obtain that $(I-C)\hat{p}=\hat{d}$. If we desire to obtain the production vector, then we multiply both sides by the Leontief inverse to obtain that $\hat{p}=(I-C)^{-1}\hat{d}$. If the external demand for industry $1$ changes, we get that

$$\hat{p}=(I-C)^{-1}(\hat{d}+e_1).$$

In our case, we first need to find this matrix $A$ from the IO table data that we have. We can do so by first dividing each row by the total industry output for that particular row and thus obtain values for each entry in that row that corespond to the percentage of that product's use for the industry in between 0 and 1. For now, I assume that this vector $\hat{d}$ can be represented as the sum of the components in the total value added secondary industries. 


## Initial Thoughts on Modeling

In our data, I believe that we would like to examine the output of the industries by the rows up to the 'Total Intermediate'. The main issue with this so far is that 'Scrap, used and secondhand goods' as well as 'Noncomparable imports and rest-of-the-world adjustments [1]' have values outside of $[0,1]$. For this, I have no clue and decided to just get rid of them. **Judgement Call here** 

# Barebones Model

The goal is to develop a basic model depicting the processes of the IO tables in a simple fashion. 

## Idea 1 - Industries before Total Intermediate

In this framework, we would like to build our analysis by removing all columns after 'Total Intermediate'. Within this idea, I would take the government industries and lump them together into a 

In [5]:
df = io_current[io_current['year'] == 2020]
col_idx = df.columns.tolist().index('Total Intermediate') - 1
row_idx = df['Name'].tolist().index('Total Intermediate') - 2
df.index = df['Name']
df = df.drop(columns=['year', 'Name'])
df = df.iloc[:row_idx, :col_idx]
A = df.iloc[:, :-1] 
A = A.div(df['Total Intermediate'] + 10).fillna(0)
print(len(df['Total Intermediate'].tolist()))
A

NameError: name 'io_current' is not defined

In [36]:
# let us look at 1997 for an example

# remove the ones after total intermediate and scrap one mentioned above
desired_col_idx = io_current[io_current['year'] == 1997]['Name'].tolist().index('Total Intermediate') - 2

# normalize the data by the total commodity output
C = io_current[[col for col in io_current.columns.tolist() if col != 'Name']][io_current['year'] == 1997].div(io_current[io_current['year'] == 1997]['Total Commodity Output'], axis=0)

# remove the columns past the 'Total Intermediate' and get rid of the 'year' column
C = C.iloc[:desired_col_idx, 1:]

# name the resulting dataframe
C.index = io_current[io_current['year'] == 1997]['Name'].tolist()[:desired_col_idx]

# remove 'Total Intermediate', 'Total Final Uses (GDP)', and 'Total Commodity Output'
C = C.drop(columns=['Total Intermediate', 'Total Final Uses (GDP)', 'Total Commodity Output'])

C

,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Wood products,Nonmetallic mineral products,Primary metals,...,Federal national defense: Gross investment in equipment,Federal national defense: Gross investment in intellectual property products,Federal national nondefense: Consumption expenditures,Federal national nondefense: Gross investment in structures,Federal national nondefense: Gross investment in equipment,Federal national nondefense: Gross investment in intellectual property products,State and local: Consumption expenditures,State and local: Gross investment in structures,State and local: Gross investment in equipment,State and local: Gross investment in intellectual property products
Farms,0.130982,0.002540,0.000000,0.000131,0.000019,0.000000,0.004990,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Forestry, fishing, and related activities",0.262125,0.186577,0.000000,0.001186,0.000000,0.000000,0.000000,0.263684,0.000022,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oil and gas extraction,0.000000,0.000000,0.121492,0.000198,0.000000,0.282960,0.000000,0.000047,0.000000,0.001375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Mining, except oil and gas",0.022759,0.000038,0.000496,0.067057,0.000267,0.147182,0.146190,0.000134,0.096265,0.180282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Support activities for mining,0.000000,0.000000,0.081860,0.026665,0.007847,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Federal general government (defense),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Federal general government (nondefense),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Federal government enterprises,0.000329,0.000214,0.000000,0.000197,0.000000,0.004651,0.000115,0.000000,0.000000,0.000444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
State and local general government,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Sources

- https://deliverypdf.ssrn.com/delivery.php?ID=142095081069091010100114080076120113014042095000089091121086081094073011121025006092119034022008009024050126007065100005115031006007037073081010101127093115018122104037082049074086104083126019115004027076089070121007094071117124116087095116120028127123&EXT=pdf&INDEX=TRUE

- Plosser: https://www.jstor.org/stable/pdf/1840430.pdf?refreqid=excelsior%3A1f614c97be9e6b0cf8359bddc9fb295a&ab_segments=&origin=

- Smirnov proof: https://arxiv.org/pdf/1910.06739.pdf

- Meeusen: https://www.jstor.org/stable/pdf/2525757.pdf

- Demirer: https://economics.sas.upenn.edu/system/files/2020-01/Demirer_JMP.pdf

- Antras harvard: https://dash.harvard.edu/bitstream/handle/1/3196325/antras_usaggregate.pdf

- io tables for dummies: https://www.uvm.edu/~wgibson/CYU/CYU_InputOutput.pdf